In [1]:
import os
import sys
import random
import shutil
import cv2
import json
from PIL import Image
from tqdm import tqdm
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import base64
import numpy as np
from pycocotools import _mask as coco_mask
from typing import Text, Dict, Tuple
import zlib

from ultralytics import YOLO


In [2]:
class config():

    seed = 42

    DATA_PATH = '/data/'
    TRAIN_PATH = DATA_PATH + 'train/'
    POLYGON_PATH = DATA_PATH + 'polygons.jsonl'
    TEST_PATH = DATA_PATH + "test"
    
    conf=0.001
    iou=0.6
    max_det=1000
    imgsz=512

In [3]:
def encode_binary_mask(mask: np.ndarray) -> Text:
    """Converts a binary mask into OID challenge encoding ascii text."""

    # check input mask --
    if mask.dtype != bool:
        raise ValueError(
            "encode_binary_mask expects a binary mask, received dtype == %s" %
            mask.dtype)

    mask = np.squeeze(mask)
    if len(mask.shape) != 2:
        raise ValueError(
            "encode_binary_mask expects a 2d mask, received shape == %s" %
            mask.shape)

    # convert input mask to expected COCO API input --
    mask_to_encode = mask.reshape(mask.shape[0], mask.shape[1], 1)
    mask_to_encode = mask_to_encode.astype(np.uint8)
    mask_to_encode = np.asfortranarray(mask_to_encode)

    # RLE encode mask --
    encoded_mask = coco_mask.encode(mask_to_encode)[0]["counts"]

    # compress and base64 encoding --
    binary_str = zlib.compress(encoded_mask, zlib.Z_BEST_COMPRESSION)
    base64_str = base64.b64encode(binary_str)
    return base64_str

In [4]:
model = YOLO("/data/yolo_32_100e.pt")

In [5]:
def dilate_predict_mask(out_mask):
    for i in range(len(out_mask)):
        kernel = np.ones(shape=(3, 3), dtype=np.uint8)
        out_mask[i] = cv2.dilate(out_mask[i], kernel, 4)
    return out_mask


def get_prediction_string(masks: list):
        prediction_string = ""
        if masks:
            for outputs in masks:
                mask = outputs["mask"]
                mask = np.where(mask > 0.5, 1, 0).astype(np.bool_)
                base64_str = encode_binary_mask(mask)
                confidence = outputs["confidence"]
                prediction_string += f"0 {confidence} {base64_str.decode('utf-8')} "
        else:
            return ""
        return prediction_string
    
    
def get_prediction(model, image_dir, img_name):
    sublist=[]
    preds = model(os.path.join(image_dir, img_name), save=False, conf=config.conf, iou=config.iou, max_det=config.max_det, imgsz=config.imgsz)
    if preds[0].masks:
        for i in range(len(preds[0].masks.data)):
            conf = round(float(preds[0].boxes.conf[i]), 2)
            mask = np.expand_dims(preds[0].masks.data[i].cpu().numpy(), axis=0).transpose(1,2,0)
            mask = dilate_predict_mask(mask)
            sublist.append({"mask": mask, "confidence": conf})
    return sublist

In [6]:
def create_submission(model, image_dir):
    img_ids = []
    heights = []
    widths = []
    prediction_string = []
    filenames = os.listdir(image_dir)
    for filename in filenames:
        img_id = filename.split(".")[0]
        sublist = get_prediction(model, image_dir, filename)
        if len(sublist) > 0: 
            pred_str = get_prediction_string(sublist)  
            height = sublist[0]['mask'].shape[0]
            width = sublist[0]['mask'].shape[1]
        else:
            pred_str = ""  
            height = 0
            width = 0
        img_ids.append(img_id)
        heights.append(height)
        widths.append(width)
        prediction_string.append(pred_str)  
        
    df = pd.DataFrame({'id': img_ids, 'height': 512, 'width': 512, 'prediction_string': prediction_string})

    return df, sublist

In [7]:
df, sublist = create_submission(model, config.TEST_PATH)


image 1/1 D:\CODE\competitions\HuBMAP\HuBMAP-solution\data\test\72e40acccadf.tif: 512x512 88 0s, 66.9ms
Speed: 1.0ms preprocess, 66.9ms inference, 32.4ms postprocess per image at shape (1, 3, 512, 512)


In [8]:
df.to_csv('submission.csv', index=False)
df.head()

,id,height,width,prediction_string
0,72e40acccadf,512,512,0 0.89 eNqLzgg0tU20N/U28jcEQwMDMGloAGcbGhhA5aA...
